In [95]:
import xlwings as xw
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import (
    dataclass,
    field,
)
from datetime import (
    datetime as dt,
)
from openpyxl.utils import (
    get_column_letter as col_let,
)
from typing import (
    List,
    Dict,
    TypeVar,
    Callable,
)

In [96]:
deskPath = Path.home() / 'Desktop'
deskPath

WindowsPath('C:/Users/424595/Desktop')

## TypeVars

In [97]:
xwBook = TypeVar(xw.main.Book)
xwSheet = TypeVar(xw.main.Sheet)
xwRange = TypeVar(xw.main.Range)

## Automating Book Selections from Open Books

#### Demonstrative DT Function

In [98]:
dt.strptime('2019 January', '%Y %B') > dt.strptime('2019 December', '%Y %B')

False

### AFV Sheets

In [99]:
def redefine_sheets(book: xwBook = xw.books.active) -> Dict[str, xwSheet]:
    '''Creates a dictionary object of the book's sheets.'''
    return {s.name: s for s in book.sheets}

In [100]:
newAFV = redefine_sheets()

In [101]:
tots = newAFV['Annual Totals']

In [102]:
@dataclass
class SheetCorners:
    '''TODO: add default values that calculate start positions'''
    sheet: xwSheet = field(repr=False)
    down: xwRange = field(init=False)
    right: xwRange = field(init=False)
        
    def __post_init__(self):
        self.down = self.sheet['A1048576'].end('up')
        self.right = self.sheet['XFD1'].end('left')
        self.downright = self.right.offset(self.down.row - 1, 0)
        self.entire_range = self.sheet['A1:' + self.downright.address]
    

# Flipping the Totals

In [103]:
# tots.book.sheets.add('Flipped Totals')

In [104]:
newAFV = redefine_sheets()

In [105]:
# flip = newAFV['Flipped Totals']

In [106]:
# tots[SheetCorners(tots).entire_range.address].api.Copy()
# newAFV['Flipped Totals']['A1'].api.PasteSpecial(
#     Paste=-4163,
#     Operation=-4142,
#     SkipBlanks=False,
#     Transpose=True,
# )

In [107]:
def _delete_flip():
    tots.book.app.display_alerts = False
    flip.api.Delete()
    tots.book.app.display_alerts = True    

### Merging/Unmerging

In [108]:
# flip['D1:A1'].api.MergeCells = True
# flip['D2:A2'].api.MergeCells = True
# flip['D3:A3'].api.MergeCells = True

# PD'd

In [109]:
# tots.book.api.SaveAs(str(deskPath / 'DropDir' / tots.book.name))

In [110]:
# flipDF = pd.read_excel(
#     tots.book.fullname,
#     header=[0],
#     index_col=[0, 1, 2, 3],
#     sheet_name='Flipped Totals'
# )

In [111]:
flipDF = pd.DataFrame(
    tots[SheetCorners(tots).entire_range.address].value
).transpose()

### Cols Renamed

In [112]:
def flip_cols():
    '''Computes the columns in flipDF with a cleaned first row.'''
    return flipDF.iloc[0, :].fillna(
        {i: 'LEVEL_' + str(i) for i in range(4)}
    ).replace(
        {'Year': 'LEVEL_3'}
    )        

In [113]:
flipDF.rename(columns=flip_cols(), inplace=True)

In [114]:
flipDF.drop(0, inplace=True)

In [115]:
flipDF['CLEAN_3'] = flipDF['LEVEL_3']
flipDF['CLEAN_3'].fillna(
    flipDF['LEVEL_2'],
    inplace=True,
)
flipDF['CLEAN_3'].fillna(
    flipDF['LEVEL_1'],
    inplace=True
)

In [116]:
flipDF['CLEAN_2'] = flipDF['LEVEL_2']
flipDF['CLEAN_2'].fillna(
    flipDF['LEVEL_1'],
    inplace=True,
)
flipDF['CLEAN_2'].fillna(
    method='ffill',
    inplace=True,
)

In [117]:
flipDF['CLEAN_1'] = flipDF['LEVEL_1']
flipDF['CLEAN_1'].fillna(
    method='ffill',
    inplace=True,
)
flipDF['CLEAN_0'] = flipDF['LEVEL_0']
flipDF['CLEAN_0'].fillna(
    method='ffill',
    inplace=True,
)

### More NaNs

In [118]:
flipDF['CLEAN_0'].fillna('(blank)', inplace=True)

In [119]:
flipDF['CLEAN_1'].fillna('(blank)', inplace=True)

In [120]:
flipDF['CLEAN_2'].fillna('(blank)', inplace=True)

### Cleaning/Filling Data

In [121]:
flipDF.drop(['TOTAL'], axis='columns', inplace=True)

In [122]:
# to lower case, strip out spaces, and delimit with undies
for i in range(4):
    flipDF['CLEAN_' + str(i)] = flipDF['CLEAN_' + str(i)].apply(
        lambda x: '_'.join(x.lower().strip().split())
    )

In [123]:
# fixing subaru
flipDF['CLEAN_2'] = flipDF['CLEAN_2'].apply(
    lambda x: x if 'subaru' not in x else 'subaru'
)

In [124]:
flipDF[
    (flipDF['CLEAN_3'].str.contains('sales', case=False))
    | (flipDF['CLEAN_3'].str.contains('share', case=False))
]

,LEVEL_0,LEVEL_1,LEVEL_2,LEVEL_3,CY 1999,CY 2000,CY 2001,CY 2002,CY 2003,CY 2004,...,CY 2015,CY 2016,CY 2017,CY 2018,CY 2019,CY 2020,CLEAN_3,CLEAN_2,CLEAN_1,CLEAN_0
1,None,None,None,Industry Sales,1.39919e+06,1.74032e+07,1.71775e+07,1.68371e+07,1.66737e+07,1.69127e+07,...,1.74705e+07,1.75558e+07,1.72307e+07,1.7264e+07,1.70475e+07,2.51733e+06,industry_sales,(blank),(blank),(blank)
2,None,None,None,Alt-Fuel Vehicle Sales,17,9367,20287,35961,47525,84195,...,500835,515824,581343,634565,717329,124544,alt-fuel_vehicle_sales,(blank),(blank),(blank)
10,None,Total US FCV Sales,None,None,None,None,None,None,None,None,...,110,1082,2313,2360,2080,231,total_us_fcv_sales,total_us_fcv_sales,total_us_fcv_sales,fuel_cell_vehicles
11,None,Cumulative US FCV Sales,None,None,None,None,None,None,None,None,...,170,1252,3565,5925,8005,8236,cumulative_us_fcv_sales,cumulative_us_fcv_sales,cumulative_us_fcv_sales,fuel_cell_vehicles
12,None,FCV Share of Alt Fuel Sales,None,None,None,None,None,None,None,None,...,0.000219633,0.00209761,0.00397872,0.00371908,0.00289965,0.00185477,fcv_share_of_alt_fuel_sales,fcv_share_of_alt_fuel_sales,fcv_share_of_alt_fuel_sales,fuel_cell_vehicles
13,None,FCV Industry Share of Sales,None,None,None,None,None,None,None,None,...,6.29633e-06,6.16322e-05,0.000134237,0.000136701,0.000122012,9.17639e-05,fcv_industry_share_of_sales,fcv_industry_share_of_sales,fcv_industry_share_of_sales,fuel_cell_vehicles
144,None,Prius Share of Total US Hybrid Sales,None,None,0,0.593787,0.766796,0.559467,0.518211,0.641249,...,0.299107,0.278686,0.173308,0.141924,0.116009,0.0890141,prius_share_of_total_us_hybrid_sales,prius_share_of_total_us_hybrid_sales,prius_share_of_total_us_hybrid_sales,hybrids
145,None,Prius v Share of Total US Hybrid Sales,None,None,0,0,0,0,0,0,...,0.0732346,0.0418592,0.0248658,0.00590802,2.24163e-05,0,prius_v_share_of_total_us_hybrid_sales,prius_v_share_of_total_us_hybrid_sales,prius_v_share_of_total_us_hybrid_sales,hybrids
146,None,Prius c Share of Total US Hybrid Sales,None,None,0,0,0,0,0,0,...,0.099585,0.0576501,0.0318915,0.0240647,0.00317814,5.01488e-05,prius_c_share_of_total_us_hybrid_sales,prius_c_share_of_total_us_hybrid_sales,prius_c_share_of_total_us_hybrid_sales,hybrids
147,None,Prius Family Share of Total US Hybrid Sales (D...,None,None,0,0.593787,0.766796,0.559467,0.518211,0.641249,...,0.471927,0.378195,0.230066,0.171897,0.11921,0.0890642,prius_family_share_of_total_us_hybrid_sales_(d...,prius_family_share_of_total_us_hybrid_sales_(d...,prius_family_share_of_total_us_hybrid_sales_(d...,hybrids


In [125]:
# fcv toyota is on level 1, nameplate on 3
# hybrid toyota is on level 3
# phev toyota is on level 2
# phev only has 3 levels; toyota on 2, and rav4 ev on 3

flipDF[
    (flipDF['CLEAN_3'].str.contains('total us', case=False))
    & (~flipDF['CLEAN_3'].str.contains('hawaii', case=False))
    & (~flipDF['CLEAN_3'].str.contains('Family'))
    | (flipDF['CLEAN_3'])
]

,LEVEL_0,LEVEL_1,LEVEL_2,LEVEL_3,CY 1999,CY 2000,CY 2001,CY 2002,CY 2003,CY 2004,...,CY 2015,CY 2016,CY 2017,CY 2018,CY 2019,CY 2020,CLEAN_3,CLEAN_2,CLEAN_1,CLEAN_0
1,None,None,None,Industry Sales,1.39919e+06,1.74032e+07,1.71775e+07,1.68371e+07,1.66737e+07,1.69127e+07,...,1.74705e+07,1.75558e+07,1.72307e+07,1.7264e+07,1.70475e+07,2.51733e+06,industry_sales,(blank),(blank),(blank)
2,None,None,None,Alt-Fuel Vehicle Sales,17,9367,20287,35961,47525,84195,...,500835,515824,581343,634565,717329,124544,alt-fuel_vehicle_sales,(blank),(blank),(blank)
3,Fuel Cell Vehicles,TOYOTA,Mirai,CA,None,None,None,None,None,None,...,63,980,1810,1657,1479,110,ca,mirai,toyota,fuel_cell_vehicles
4,None,None,None,NE,None,None,None,None,None,None,...,None,None,None,None,None,None,ne,mirai,toyota,fuel_cell_vehicles
5,None,None,None,Fleet (Reference),None,None,None,None,None,None,...,9,54,28,35,14,4,fleet_(reference),mirai,toyota,fuel_cell_vehicles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,None,None,None,Fiat Chrysler Combined Alt-Fuel Sales,0,0,0,0,0,0,...,4011,5330,10600,5415,2825,412,fiat_chrysler_combined_alt-fuel_sales,electric_share_of_us_industry,electric_share_of_us_industry,annual_(cy)_us_afv_sales_by_mfg_-_condensed
248,None,None,None,Mitsubishi Combined Alt-Fuel Sales,0,0,0,0,0,0,...,115,94,105,4166,2810,246,mitsubishi_combined_alt-fuel_sales,electric_share_of_us_industry,electric_share_of_us_industry,annual_(cy)_us_afv_sales_by_mfg_-_condensed
249,None,None,None,Jaguar Combined Alt-Fuel Sales,None,None,None,None,None,None,...,None,None,None,393,3001,568,jaguar_combined_alt-fuel_sales,electric_share_of_us_industry,electric_share_of_us_industry,annual_(cy)_us_afv_sales_by_mfg_-_condensed
250,None,None,None,Tesla Combined Alt-Fuel Sales,0,0,0,0,0,0,...,25914,47644,44352,115850,178950,42800,tesla_combined_alt-fuel_sales,electric_share_of_us_industry,electric_share_of_us_industry,annual_(cy)_us_afv_sales_by_mfg_-_condensed


# To Excel...

In [126]:
try:    
    tots.book.sheets.add('EVC-AS Charting')
except ValueError:
    print('sheet already exists')
    pass

sheet already exists


In [127]:
newAFV = redefine_sheets()

In [128]:
evc = newAFV['EVC-AS Charting']

In [129]:
evc.api.Tab.Color = xw.constants.RgbColor.rgbDeepSkyBlue

In [130]:
# some formulas to paste in the sheet
b1 = '=SUBSTITUTE(SUBSTITUTE(MID(CELL("filename",A1),FIND("[",CELL'
b1 +='("filename",A1))+1,FIND("]",CELL("filename",A1))-FIND("[",CELL("filename",A1))-1)'
b1 += ',"US Alt-Fuel Sales History To Date ",""),".xlsx","")&" HV/PHV/BEV Sales and Gasoline Price"'

b4 = '=LEFT(B1,FIND(" ",B1,FIND(" ",B1)+1)-1)'

b5 = '="*"&LEFT(B1, 4)&" sales data is year to date"&CHAR(10)&LEFT(B1,4)&"'
b5 +=' gasoline price reflects YTD average"&CHAR(10)&"All other gas prices reflect annual averages"'

b6 = '="Sources:"&CHAR(10)&"eia.gov"&CHAR(10)&"VMC Alt-Fuel Sales'
b6 += 'History to Date "&LEFT(B1,FIND(" ",B1,FIND(" ",B1)+1)-1)'

evc['B1'].value = b1
evc['B4'].value = b4
evc['B5'].value = b5
evc['B6'].value = b6

In [131]:
# pasting years
evc['D1'].value = tuple(
    i if i != dt.now().year else str(i) + '*' 
    for i in range(2010, dt.now().year + 1)
)

In [132]:
# total industry sales
evc['D2'].value = (
    flipDF[flipDF['CLEAN_3'] == 'industry_sales'][
        ['CY ' + str(i) for i in range(2010, dt.now().year + 1)]
    ]
    .iloc[0]
    .to_list()
)

In [133]:
# total hev sales
evc['D3'].value = (
    flipDF[flipDF['CLEAN_1'] == 'total_us_hybrid_sales'][
        ['CY ' + str(i) for i in range(2010, dt.now().year + 1)]
    ]
    .iloc[0]
    .to_list()
)

In [134]:
# total phev sales
evc['D4'].value = (
    flipDF[flipDF['CLEAN_1'] == 'total_us_plug-in_sales'][
        ['CY ' + str(i) for i in range(2010, dt.now().year + 1)]
    ]
    .iloc[0]
    .to_list()
)

In [135]:
# total bev sales
evc['D5'].value = (
    flipDF[flipDF['CLEAN_1'] == 'total_us_electric_sales'][
        ['CY ' + str(i) for i in range(2010, dt.now().year + 1)]
    ]
    .iloc[0]
    .to_list()
)

In [136]:
# total fcev sales
evc['D6'].value = (
    flipDF[flipDF['CLEAN_1'] == 'total_us_fcv_sales'][
        ['CY ' + str(i) for i in range(2010, dt.now().year + 1)]
    ]
    .iloc[0]
    .to_list()
)

In [137]:
# record titles
evc['C2'].options(transpose=True).value = (
    'Industry Sales',
    'HEV Total',
    'PHEV Total',
    'BEV Total',
    'FCEV Total',
)

In [138]:
# adding percents
for cell in evc['D9:' + SheetCorners(evc).right.offset(11, 0).address]:
    letter = col_let(cell.column)
    cell.value = '=' + letter + str(cell.row - 6) + '/' + letter + '$2'
    
# number format percents
evc['D9:N12'].number_format = '0.00%'

In [139]:
# record titles
evc['C9'].options(transpose=True).value = tuple(
    map(lambda x: x.replace('Total', 'Industry %'), evc['C3:C6'].value)
)

In [140]:
evc['C14'].value = 'U.S. Regular Conventional Retail Gasoline Prices ($/gallon)*'

## HEVs

In [141]:
hevs = list(
    flipDF[
        (flipDF['CLEAN_0'] == 'hybrids')
        & (flipDF['CLEAN_1'] == 'competition')
    ]['CLEAN_2'].unique()
)

In [142]:
hevs.append('toyota_/_lexus')

In [143]:
evc['C19'].options(transpose=True).value = sorted(hevs)

In [144]:
def sum_comp_hev_year(oem: str, year: int) -> int:
    '''Given an oem and a year, returns the sum of that oem's hybrid sales for that year.'''
    year = 'CY ' + str(year)
    return flipDF[
        (flipDF['CLEAN_0'] == 'hybrids')
        & (flipDF['CLEAN_2'] == oem)
    ][year].sum()

In [145]:
def sum_toyota_hev_year(nix: str, year: int) -> int:
    '''Returns toyota hybrid sales for the given year'''
    year = 'CY ' + str(year)
    return flipDF[
        (flipDF['CLEAN_0'] == 'hybrids')
        & (flipDF['CLEAN_1'] == 'toyota_/_lexus')
        & (flipDF['CLEAN_3'].str.contains('total_us'))
        & (~flipDF['CLEAN_3'].str.contains('hawaii'))
        & (flipDF['CLEAN_3'] != ('total_us_prius_family'))
    ][year].sum()

In [146]:
def sum_years(oem: str, _: Callable[[str, int], int]):
    return tuple(_(oem, year) for year in range(2010, dt.now().year + 1))

In [147]:
for cell in evc['C19:C' + str(evc['C19'].end('down').row)]:
    if cell.value != 'toyota_/_lexus':
        cell.offset(0, 1).value = sum_years(cell.value, sum_comp_hev_year)
    else:
        cell.offset(0, 1).value = sum_years(cell.value, sum_toyota_hev_year)
    cell.value = cell.value.upper().replace('_', ' ')

In [148]:
hevp_start: str = evc['C' + str(evc['C19'].end('down').row)].offset(2, 0).address

In [149]:
evc[hevp_start].options(transpose=True).value = tuple(
    map(lambda x: x.upper().replace('_', ' '), sorted(hevs))
)

In [150]:
def calculate_percent_area(oems: List[str], start_address: str) -> str:
    '''Calculates the area of
    start_address must be an absolute excel cell address.
    
    >>> calculate_percent_area(hevs, '$D$19')
    '$D$19:$D$29'
    
    '''
    #TODO: rename this sucker
    end_address = start_address.split('$')[1] + '$' +  str(
        len(oems) - 1 + int(start_address.split('$')[-1])
    )
    return f'{start_address}:${end_address}'

In [151]:
for cell in evc['D19:' 
                + col_let(evc['D19'].end('right').column) 
                + str(evc['D19'].end('down').row)]:
    
    pcell: xwRange = cell.offset(len(hevs) + 1, 0)
    cur_col: str = col_let(cell.column)
    sum_area: str = calculate_percent_area(hevs, '$' + cur_col + '$19')
    
    pcell.value = '=' + cell.address + f'/SUM({sum_area})'
    pcell.number_format = '0%'

## PHEVs

In [ ]:
# fcv toyota is on level 1, nameplate on 3
# hybrid toyota is on level 3
# phev toyota is on level 2
# bev only has 3 levels; toyota on 2, and rav4 ev on 3

In [94]:
def sum_phev_toyota_year(nix: str, year: int) -> int:
    year = 'CY ' + str(year)
    
    flipDF[(flipDF['CLEAN_0'] == 'plug-in_hybrids')
       & (flipDF['CLEAN_3'] == 'total_us_prius_plug-in')
]

,LEVEL_0,LEVEL_1,LEVEL_2,LEVEL_3,CY 1999,CY 2000,CY 2001,CY 2002,CY 2003,CY 2004,...,CY 2015,CY 2016,CY 2017,CY 2018,CY 2019,CY 2020,CLEAN_3,CLEAN_2,CLEAN_1,CLEAN_0
156,None,None,Total US Prius Plug-in,None,0,0,0,0,0,0,...,4191,2475,20936,27595,21856,3293,total_us_prius_plug-in,total_us_prius_plug-in,prius_prime_(prius_plug_in_before_11/16),plug-in_hybrids


In [66]:
with pd.option_context('display.max_rows', 270):
    display(flipDF)

,LEVEL_0,LEVEL_1,LEVEL_2,LEVEL_3,CY 1999,CY 2000,CY 2001,CY 2002,CY 2003,CY 2004,...,CY 2015,CY 2016,CY 2017,CY 2018,CY 2019,CY 2020,CLEAN_3,CLEAN_2,CLEAN_1,CLEAN_0
1,None,None,None,Industry Sales,1.39919e+06,1.74032e+07,1.71775e+07,1.68371e+07,1.66737e+07,1.69127e+07,...,1.74705e+07,1.75558e+07,1.72307e+07,1.7264e+07,1.70475e+07,2.51733e+06,industry_sales,(blank),(blank),(blank)
2,None,None,None,Alt-Fuel Vehicle Sales,17,9367,20287,35961,47525,84195,...,500835,515824,581343,634565,717329,124544,alt-fuel_vehicle_sales,(blank),(blank),(blank)
3,Fuel Cell Vehicles,TOYOTA,Mirai,CA,None,None,None,None,None,None,...,63,980,1810,1657,1479,110,ca,mirai,toyota,fuel_cell_vehicles
4,None,None,None,NE,None,None,None,None,None,None,...,None,None,None,None,None,None,ne,mirai,toyota,fuel_cell_vehicles
5,None,None,None,Fleet (Reference),None,None,None,None,None,None,...,9,54,28,35,14,4,fleet_(reference),mirai,toyota,fuel_cell_vehicles
6,None,None,None,Total US,None,None,None,None,None,None,...,72,1034,1838,1692,1493,114,total_us,mirai,toyota,fuel_cell_vehicles
7,None,COMPETITION,None,Honda Clarity FCV,None,None,None,None,None,None,...,0,8,447,624,320,73,honda_clarity_fcv,competition,competition,fuel_cell_vehicles
8,None,None,None,Hyundai Tucson FCV,None,None,None,None,None,None,...,38,40,28,36,0,0,hyundai_tucson_fcv,competition,competition,fuel_cell_vehicles
9,None,None,None,Hyundai Nexo,None,None,None,None,None,None,...,None,None,None,8,267,44,hyundai_nexo,competition,competition,fuel_cell_vehicles
10,None,Total US FCV Sales,None,None,None,None,None,None,None,None,...,110,1082,2313,2360,2080,231,total_us_fcv_sales,total_us_fcv_sales,total_us_fcv_sales,fuel_cell_vehicles
